In [2]:
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.optimizers.legacy import Adam
from keras.preprocessing.image import ImageDataGenerator

In [3]:
train_data_gen = ImageDataGenerator(rescale=1./255)
validation_data_gen = ImageDataGenerator(rescale=1./255)

In [4]:
train_generator = train_data_gen.flow_from_directory(
        'data/train',
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

Found 28709 images belonging to 7 classes.


In [5]:
validation_generator = validation_data_gen.flow_from_directory(
        'data/test',
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

Found 7178 images belonging to 7 classes.


In [6]:
emotion_model = Sequential()

emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))

In [7]:
emotion_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 46, 46, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 44, 44, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2  (None, 22, 22, 64)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 22, 22, 64)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 20, 20, 128)       73856     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 10, 10, 128)       0         
 g2D)                                                   

In [8]:
cv2.ocl.setUseOpenCL(False)

emotion_model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.0001, decay=1e-6), metrics=['accuracy'])

C:\Users\sayed\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\optimizers\legacy\adam.py:118: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [9]:
emotion_model_info = emotion_model.fit_generator(
        train_generator,
        steps_per_epoch=28709 // 64,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=7178 // 64)

Epoch 1/50


C:\Users\sayed\AppData\Local\Temp\ipykernel_17976\1151617337.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  emotion_model_info = emotion_model.fit_generator(


448/448 [==============================] - 60s 133ms/step - loss: 1.8059 - accuracy: 0.2585 - val_loss: 1.7298 - val_accuracy: 0.3274
Epoch 2/50
448/448 [==============================] - 59s 131ms/step - loss: 1.6503 - accuracy: 0.3543 - val_loss: 1.5833 - val_accuracy: 0.4028
Epoch 3/50
448/448 [==============================] - 58s 129ms/step - loss: 1.5495 - accuracy: 0.3993 - val_loss: 1.4835 - val_accuracy: 0.4297
Epoch 4/50
448/448 [==============================] - 59s 132ms/step - loss: 1.4797 - accuracy: 0.4308 - val_loss: 1.4176 - val_accuracy: 0.4594
Epoch 5/50
448/448 [==============================] - 60s 133ms/step - loss: 1.4155 - accuracy: 0.4594 - val_loss: 1.3673 - val_accuracy: 0.4805
Epoch 6/50
448/448 [==============================] - 59s 131ms/step - loss: 1.3624 - accuracy: 0.4811 - val_loss: 1.3195 - val_accuracy: 0.4992
Epoch 7/50
448/448 [==============================] - 58s 130ms/step - loss: 1.3120 - accuracy: 0.5017 - val_loss: 1.2914 - val_accuracy: 0.5

In [10]:
model_json = emotion_model.to_json()
with open("emotion_model.json", "w") as json_file:
    json_file.write(model_json)

In [11]:
emotion_model.save_weights('emotion_model.h5')